In [2]:
import pandas as pd
import numpy as np

In [4]:
TrainData = pd.read_csv('TrainData.csv', encoding = "ISO-8859-1" )

TestData = pd.read_csv('TestData.csv', encoding = "ISO-8859-1")


In [5]:
AirCanadaTest = pd.read_csv('AirCanada.csv', encoding = "ISO-8859-1")

In [6]:
AirCanadaTest.nunique()

Review       151
Sentiment      2
dtype: int64

In [7]:
TrainData['Sentiment'].unique()

array(['Positive', 'Negative', 'Negative '], dtype=object)

In [8]:
TestData.nunique()

Review       280
Sentiment      2
dtype: int64

In [9]:
TrainData['Sentiment'] = TrainData['Sentiment'].fillna(TrainData['Sentiment'].mode()[0])

In [10]:
TrainData.Sentiment.replace({"Negative ":"Negative"},inplace=True)

In [11]:
TrainData['Sentiment'].unique()

array(['Positive', 'Negative'], dtype=object)

In [12]:
Data = pd.concat([TrainData,AirCanadaTest,TestData],axis=0,sort=False,ignore_index=True)

In [13]:
Data.shape

(785, 2)

In [14]:
# build train and test datasets
reviews = Data['Review'].values
sentiments = Data['Sentiment'].values

train_reviews = reviews[:450]
train_sentiments = sentiments[:450]

test_reviews = reviews[450:]
test_sentiments = sentiments[450:]

# Text Wrangling and Normalization

In [15]:
from bs4 import BeautifulSoup
import re
import tqdm
import unicodedata

In [16]:
#function defined to remove html tags from data

def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

In [17]:
#function defined to remove unicode data

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

In [18]:
# lower case and remove special characters\whitespaces

def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    #doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    norm_docs.append(doc)
  
  return norm_docs

In [19]:
#Data_Preprocessing

norm_train_reviews = pre_process_corpus(train_reviews)
norm_test_reviews = pre_process_corpus(test_reviews)

100%|██████████| 335/335 [00:00<00:00, 3662.79it/s]


In [20]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
# build BOW features on train reviews
cv = CountVectorizer(binary=False, min_df=5, max_df=1.0, ngram_range=(1,2))
cv_train_features = cv.fit_transform(norm_train_reviews)
cv_test_features = cv.transform(norm_test_reviews)

In [22]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [23]:

def deep_lstm():
    model = Sequential()
    model.add(LSTM(20, input_shape = (49,1), return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = True))
    model.add(LSTM(20, return_sequences = False))
    model.add(Dense(46))
    model.add(Activation('softmax'))
    
    adam = optimizers.Adam(lr = 0.001)
    model.compile(loss = 'categorical_crossentropy', optimizer = adam, metrics = ['accuracy'])
    
    return model

In [27]:
cv_train_features.shape

(450, 925)

In [28]:
cv_test_features.shape

(335, 925)

In [30]:
X_train = np.reshape(norm_train_reviews, (norm_train_reviews.shape[0], 1, norm_train_reviews.shape[1]))
X_test = np.reshape(norm_test_reviews, (norm_test_reviews.shape[0], 1, norm_test_reviews.shape[1]))

AttributeError: 'list' object has no attribute 'shape'

In [29]:
model = KerasClassifier(build_fn = deep_lstm, epochs = 200, batch_size = 50, verbose = 1)
model.fit(norm_train_reviews, train_sentiments)

AttributeError: 'str' object has no attribute 'ndim'